In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Import Customer Data
df_cust= pd.read_csv("/Users/sarahboller/Desktop/CF 4 Instacart Basket Analysis/02 Data/Original Data/customers.csv")


In [3]:
# Check Data
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [4]:
df_cust.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [5]:
df_cust.shape

(206209, 10)

In [7]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [8]:
# Check columns for inconsistancies
df_cust['STATE'].value_counts(dropna = False)


STATE
Florida                 4044
Colorado                4044
Illinois                4044
Alabama                 4044
District of Columbia    4044
Hawaii                  4044
Arizona                 4044
Connecticut             4044
California              4044
Indiana                 4044
Arkansas                4044
Alaska                  4044
Delaware                4044
Iowa                    4044
Idaho                   4044
Georgia                 4044
Wyoming                 4043
Mississippi             4043
Oklahoma                4043
Utah                    4043
New Hampshire           4043
Kentucky                4043
Maryland                4043
Rhode Island            4043
Massachusetts           4043
Michigan                4043
New Jersey              4043
Kansas                  4043
South Dakota            4043
Minnesota               4043
Tennessee               4043
New York                4043
Washington              4043
Louisiana               4043
Montana 

In [9]:
df_cust ['Gender'].value_counts(dropna = False)

Gender
Male      104067
Female    102142
Name: count, dtype: int64

In [10]:
df_cust['fam_status'].value_counts(dropna = False)

fam_status
married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: count, dtype: int64

In [11]:
df_cust['Age'].value_counts(dropna = False)

Age
19    3329
55    3317
51    3317
56    3306
32    3305
      ... 
65    3145
25    3127
66    3114
50    3102
36    3101
Name: count, Length: 64, dtype: int64

In [12]:
# Check for mixed data types
for col in df_cust.columns.tolist():
  weird = (df_cust[[col]].map(type) != df_cust[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_cust[weird]) > 0:
    print (col)

First Name


In [13]:
# Change data type
df_cust['First Name'] = df_cust['First Name'].astype('str')

In [14]:
# Check for missing values
df_cust.isnull().sum()

user_id         0
First Name      0
Surnam          0
Gender          0
STATE           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [15]:
# Check for dups
df_dups = df_cust[df_cust.duplicated()]

In [16]:
df_dups

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income


In [17]:
# Import ords prods data
ords_prods_merge = pd.read_pickle("/Users/sarahboller/Desktop/CF 4 Instacart Basket Analysis/02 Data/Prepared Data/orders_products_combined.pkl")

In [18]:
ords_prods_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 23 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   Unnamed: 0_x            int64   
 1   order_id                int64   
 2   user_id                 int64   
 3   order_number            int64   
 4   orders_day_of_week      int64   
 5   time_of_order_hr        int64   
 6   days_since_prior_order  float64 
 7   returning_customer      bool    
 8   product_id              int64   
 9   add_to_cart_order       int64   
 10  reordered               int64   
 11  Unnamed: 0_y            int64   
 12  product_name            object  
 13  aisle_id                int64   
 14  department_id           int64   
 15  prices                  float64 
 16  _merge                  category
 17  max_order               int64   
 18  loyalty_flag            object  
 19  mean_price              float64 
 20  Spender_flag            object  
 21  median

In [19]:
# Deleting previous merge column
ords_prods_merge = ords_prods_merge.drop(columns = ['_merge'])

In [20]:
# Merge ords/prods data with customer data (with indicator)
df_merged = df_cust.merge(ords_prods_merge, on = ['user_id'],indicator = True)

In [21]:
df_merged.head(20)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income,...,aisle_id,department_id,prices,max_order,loyalty_flag,mean_price,Spender_flag,median_freq,freq_flag,_merge
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,77,7,9.0,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,77,7,9.0,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,77,7,9.0,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,120,16,12.6,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,120,16,12.6,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
5,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,120,16,12.6,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
6,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,120,16,12.6,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
7,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,120,16,12.6,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
8,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,120,16,12.6,8,New customer,7.988889,Low Spender,19.0,Regular customer,both
9,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,32,4,4.3,8,New customer,7.988889,Low Spender,19.0,Regular customer,both


In [22]:
df_merged.shape

(32404859, 32)

In [23]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32404859 entries, 0 to 32404858
Data columns (total 32 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   user_id                 int64   
 1   First Name              object  
 2   Surnam                  object  
 3   Gender                  object  
 4   STATE                   object  
 5   Age                     int64   
 6   date_joined             object  
 7   n_dependants            int64   
 8   fam_status              object  
 9   income                  int64   
 10  Unnamed: 0_x            int64   
 11  order_id                int64   
 12  order_number            int64   
 13  orders_day_of_week      int64   
 14  time_of_order_hr        int64   
 15  days_since_prior_order  float64 
 16  returning_customer      bool    
 17  product_id              int64   
 18  add_to_cart_order       int64   
 19  reordered               int64   
 20  Unnamed: 0_y            int64   
 21  produc

In [25]:
df_merged['_merge'].value_counts(dropna = False)

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [26]:
# Export
path = r'/Users/sarahboller/Desktop/CF 4 Instacart Basket Analysis'

In [27]:
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'df_merged_all.pkl'))